In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mping
from moviepy.editor import VideoFileClip

def camera_calibration(nx,ny,basepath):
    objp = np.zeros((nx*ny,3),float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
    
    objpoints = [] # 3d points in world space
    imgpoints = [] # 2d points in image space
    
    # make a list of calibration images
    images = glob.glob(path.join(basepath,'calibration*.jpg'))
    
    
    for fname in images:
        img = mping.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        
        ret,corners = cv2.findChessboardCorners(gray,(nx,ny),None)
        
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            
            img = cv2.drawChessboardCorners(img,(nx,ny),corners,ret)
            cv2.imshow('input image',img)
            #cv2.waitKey(500)
            
    img_size = (img.shape[1],img.shape[0])
    ret,mtx,dist,rvecs,tvecs = cv2.calibrateCamera(objpoints,imgpoints,img_size,None,None)
        
            
    return mtx,dist

def cal_undistort(imgpath,mtx,dist):
    
    img = mping.imread(imgpath)
    
    undist = cv2.undistort(img,mtx,dist,None,mtx)
    
    
    return undist

def corners_unwarp(img,nx,ny,mtx,undist):
    gray = cv2.cvtColor(undist,cv2.COLOR_BGR2GRAY)
    
    ret,corners = cv2.findChessboardCorners(gray,(nx,ny),None)
    
    if ret == True:
        cv2.drawChessboardCorners(undist,(nx,ny),corners,ret)
        
        offset = 100
        
        img_size = (gray.shape[1],gray.shape[0])
        
        src = np.flaot32([corners[0],corners[nx - 1],corners[-1],corners[-nx]])
        
        dst = np.float32([[offset,offset],[img_size[0] - offset,offset],[img_size[0] - offset,img_size[1] - offset],[offset,img_size[1] - offset]])
        
        M = cv2.getPerspectiveTransform(src,dst)
        
        warped = cv2.warpPerspective(undist,M,img_size)
        
    return warped,M



In [ ]:
left_lane = Line()
right_lane = Line()
frame_width = 1280
frame_height = 720

LINEWIDTH = 3.7
input_scale = 4
output_frame_scale = 4
N = 4

x = [194,1117,705,575]
y = [719,719,461,461]
X = [290,990,990,290]
Y = [719,719,0,0]

src = np.floor(np.float32([[x[0],y[0]],[x[1],y[1]],[x[2],y[2]],[x[3],y[3]]])/input_scale)
dst = np.floor(np.float32([[X[0],Y[0]],[X[1],Y[1]],[X[2],Y[2]],[X[3],Y[3]]])/input_scale)

M = cv2.getPerspectiveTransform(src,dst)
M_inv = cv2.getPerspectiveTransform(dst,src)

'''
X_b = [574, 706, 706, 574]
Y_b = [719, 719, 0, 0]
src_ = np.floor(np.float32([[x[0], y[0]], [x[1], y[1]],[x[2], y[2]], [x[3], y[3]]]) / (input_scale*2))
dst_ = np.floor(np.float32([[X_b[0], Y_b[0]], [X_b[1], Y_b[1]],[X_b[2], Y_b[2]], [X_b[3], Y_b[3]]]) / (input_scale*2))
M_b = cv2.getPerspectiveTransform(src_, dst_)

'''


In [ ]:
class Line():
    def _init_(self):
        
        self.detected = False
        self.recent_xfitted = []
        self.bestx = None
        self.best_fit = None
        self.current_fit = [np.array([False])]
        self.radius_of_curvature = None
        self.line_base_pos = None
        self.diffs = np.array([0,0,0],dtype = 'float')
        self.allx = None
        self.ally = None
        
    def average_pre_lanes(self):
        tmp = copy(self.prev_fitx)
        tmp.append(self.cur_fitx)
        self.mean_fitx = np.mean(tmp,axis = 0)
        
    def append_fitx(self):
        if len(self.prev_fitx) == N:
            self.prev_fitx.pop(0)
        self.prev_fitx.append(self.mean_fitx)
        
    def process(self,ploty):
        self.cur_fity = ploty
        self.average_pre_lanes()
        self.append_fitx()
        self.prev_poly = self.current_poly
        

def abs_sobel_thresh(img,orient = 'x',thresh_min = 0,thresh_max = 255):
    
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray,cv2.CV64F,1,0,ksize = sobel_kernel))
        
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel))
        
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min)&(scaled_sobel<=thresh_max)] = 1
    
    return binary_output

def mag_thresh(img,sobel_kernel = 3,mag_thresh = (0,255)):
    
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    
    return binary_output

def dir_threshold(img,sobel_kernel = 3, thresh = (0,np.pi/2)):
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    
    absgraddir = np.arctan2(np.absolute(sobely),np.absolute(sobelx))
    
    binary_output = np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    
    return binary_output

def find_edges(img,s_thresh = (170,255),sx_thresh = (20,100),dir_thresh = (0,np.pi/2)):
    
    img = np.copy(img)
    hls = cv2.cvtColor(img,cv2.COLOR_RGB2HLS).astype(np.float)
    
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0])&(s_channel <= s_thresh[1])] = 1
    
    sxbinary = abs_sobel_thresh(img,orient = 'x',thresh_min = 0,thresh_max = 255)
    
    dir_binary = dir_threshold(img,sobel_kernel = 3, thresh = dir_thresh)
    
    combined_binary = np.zeros_like(s_channel)
    combined_binary[((sxbinary == 1) & (dir_binary == 1)) | ((s_binary == 1) & (dir_binary == 1))] = 1
    
    #color_binary = np.dstack((np.zeros_like(sxbinary),sxbinary,s_binary))*255
    
    '''
    
    # add more weights for the s channel
    c_bi = np.zeros_like(s_channel)
    c_bi[( (sxbinary == 1) & (s_binary==1) )] = 2

    ave_binary = (combined_binary + c_bi)
    
    '''
    
    return combined_binary
'''
def window_mask(width,height,img_ref,center,level):
    
    output = np.zeros_like(img_ref)
    output[] = 1
    
    return output
'''

def warper(img,M):
    
    img_size = (img.shape[1],img.shape[0])
    warped = cv2.warpPerspective(img,M,img_size,flags = cv2.INTER_NEAREST)
    
    return warped

def lane_locating(binary_warped,visualization = False):
    
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:],axis = 0)
    
    out_img = np.dstack((binary_warped,binary_warped,binary_warped))*255
    
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    nwindows = 9
    window_height = np.int(binary_warped.shape[0]/nwindows)
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    margin = 100
    minpix = 50
    
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(nwindows):
        
        win_y_low = binary_warped.shape[0] - (windows + 1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0),2)
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0),2)
        
        good_left_inds = ((nonzeroy >= win_y_low)&(nonzeroy < win_y_high)&(nonzerox >= win_xleft_low)&(nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low)&(nonzeroy < win_y_high)&(nonzerox >= win_xright_low)&(nonzerox < win_xright_high)).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    leftx = nonzeros[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)
    
    if visualization:
        
        ploty = np.linspace(0,binary_warped.shape[0] - 1,binary_warped.shape[0])
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
        out_img[nonzeroy[left_lane_inds],nonzerox[left_lane_inds]] = [255,0,0]
        out_img[nonzeroy[right_lane_inds],nonzerox[right_lane_inds]] = [0,0,255]
        
        plt.imshow(out_img)
        plt.plot(left_fitx,ploty,color = 'yellow')
        plt.plot(right_fitx,ploty,color = 'yellow')
        plt.xlim(0,1280)
        plt.ylim(720,0)
        plt.show()
        
    return left_fit,right_fit

def window_search(left_fit, right_fit, binary_warped, margin=100, visualization=False):
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin))&(nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin))&(nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)
    
    ploty = np.linspace(0,binary_warped.shape[0] - 1,binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    if visualization:
        out_img = np.dstack((binary_warped,binary_warped,binary_warped))*255
        window_img = np.zeros_like(out_img)
        
        out_img[nonzeroy[left_lane_inds],nonzerox[left_lane_inds]] = [255,0,0]
        out_img[nonzeroy[right_lane_inds],nonzerox[right_lane_inds]] = [0,0,255]
        
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx - margin,ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx + margin,ploty])))])
        left_line_pts = np.hstack((left_line_window1,left_line_window2))
        
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx - margin,ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx + margin,ploty])))])
        right_line_pts = np.hstack((right_line_window1,right_line_window2))
        
        cv2.fillpoly(window_img,np.int_([left_line_pts]),(0,255,0))
        cv2.fillpoly(window_img,np.int_([right_line_pts]),(0,255,0))
        result = cv2.addweighted(out_img,1,window_img,0.3,0)
        plt.imshow(result)
        plt.plot(left_fitx,ploty,color = 'yellow')
        plt.plot(right_fitx,ploty,color = 'yellow')
        plt.xlim(0,1280)
        plt.ylim(720,0)
    
    return left_fit,right_fit
            
'''    
    
def find_window_centroids(image,window_width,window_height,margin):
    
    window_centroids = []
    window = np.ones(window_width)
    
    l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)],axis = 0)
    l_center = np.argmax(np.convolve(window,l_sum)) - window_width/2
    r_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)],axis = 0)
    r_center = np.argmax(np.convolve(window,r_sum)) - window_width/2 + int(image.shape[1]/2)
    
    window_centroids.append((l_center,r_center))
    
    for level in range(1,(int)(image.shape[0]/window_height)):
        image_layer = np.sum(image[int(image.shape[0] - (level + 1)*window_height):int(image.shape[0] - level*window_height),:],axis = 0)
        cov_signal = np.convolve(window,image_layer)
        offset = window_width/2
        l_min_index = int(max(l_center + offset - margin,0))
        l_max_index = int(min(l_center + offset + margin,image.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index]) + l_min_index - offset
        
        r_min_index = int(max(r_center + offset - margin,0))
        r_max_index = int(min(r_center + offset + margin,image.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index]) + r_min_index - offset
        
    return window_centroids

'''

def curvature(ploty,leftx,rightx,visualization = False):

    leftx = leftx[::-1]
    rightx = rightx[::-1]
    
    left_fit = mp.polyfit(ploty,leftx,2)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fit = np.polyfit(ploty,rightx,2)
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    mark_size = 3
    plt.plot(leftx,ploty,'o',color = 'red',marksize = mark_size)
    plt.plot(rightx,ploty,'o',color = 'blue',marksize = mark_size)
    plt.xlim(0,1280)
    plt.ylim(0,720)
    plt.plot(left_fitx,ploty,color = 'green', linewidth = 3)
    plt.plot(right_fitx,ploty,color = 'green', linewidth = 3)
    plt.gca().invert_yaxis()

    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5)/np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5)/np.absolute(2*right_fit[0])


    ym_per_pix = 30/720
    xm_per_pix = 3.7/700

    left_fit_cr = np.polyfit(ploty*ym_per_pix,leftx*xm_per_pix,2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix,rightx*xm_per_pix,2)

    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5)/np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5)/np.absolute(2*right_fit_cr[0])   

    return left_curverad,right_curverad

def car_back_center(ploty,left_fitx,right_fitx,undist):
    
    height = undist.shape[0]
    width = undist.shape[1]
    
    pts_left = np.array([np.tranpose(np.vstack([left_fitx,ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx,ploty])))])
    pts = np.hstack((pts_left,pts_right))
    
    bottom_l = left_fitx[height - 1]
    bottom_r = right_fitx[0]
    
    left_position = width/2.0 - bottom_l
    right_position = bottom_r - width/2.0
    width = bottom_r - bottom_l
    
    if left_position >= right_position:
        offset = left_position/width*LINEWIDTH - LINEWIDTH/2.0
    else:
        offset = LINEWIDTH/2.0 - right_position/width*LINEWIDTH
        
    return offset,pts

def drawing(binary_warped,ploty,left_fitx,right_fitx):
    
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero,warp_zero,warp_zero))

    pts_left = np.array([np.transpose(np.vstack([left_fitx,ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx,ploty])))])

    cv2.fillPoly(color_warp,np.int_([pts]),(0,255,0))

    newwarp = cv2.warpPerspective(color_warp,Minv,(image.shape[1],image.shape[0]))

    result = cv2.addWeighted(undist,1,newarp,0.3,0)
    
    return result

def process_imgae(img,visulization = False):
    
    undist = cv2.undistort(img,mtx,dist,None,mtx)
    undist = cv2.resize(undist,(0,0),fx = 1/input_scale,fy = 1/input_scale)
    
    img_binary = find_edges(undist)
    
    binary_warped = warper(img_binary,M)
    binary_sub = np.zeros_like(binary_warped)
    binary_sub[:,int(80):int(-20)] = binary_warped[:,int(80):int(-20)]
    
    ploty = np.linspace(0,binary_warped.shape[0] - 1,binary_warped.shape[0])
    
    if left_lane.detected:
        tracker(binary_sub,ploty,visualization)
    else:
        detector(binary_sub,ploty,visualization)
    
    left_lane.process(ploty)
    right_lane.process(ploty)
    
    left_curverad,right_curverad = curvature(ploty,left_lane.mean_fitx,right_lane.mean_fitx)
    
    
    
    
    
    return result


In [ ]:
# image processing
imagepath = './test_images/test1.jpg'

img = mping.imread(imagepath)

img_aug = process_image(img)

f,(ax1,ax2) = plt.subplots(1,2,figsize = (25,9))

ax1.imshow(img)
ax1.set_title('Original Image', fontsize = 30)
ax2.imshow(img_aug)
ax2.set_title('Augmented Image', fontsize = 30)

plt.show()




In [ ]:
# video processing

video_output = './project_video_aug.mp4'
clip1 = VideoFileClip('./projet_video.mp4')

clip = clip1.fl_image(process_image)
clip.write_videofile(video_output,audio = False)